# MLE of Normal parameters

[Dataset download](https://s3.amazonaws.com/bebi103.caltech.edu/data/good_invitro_droplet_data.csv)

<hr>

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

In [2]:
import numpy as np
import pandas as pd

<hr>

Recall our first model for spindle length.

\begin{align}
l_i \sim \text{Norm}(\phi, \sigma) \;\;\forall i.
\end{align}

For this model, we have already worked out analytically that the maximum likelihood estimates for the parameters $\phi$ and $\sigma$ are given by their respective plug-in estimates.

\begin{align}
&\phi^* = \hat{\phi}\\[1em]
&\sigma^* = \hat{\sigma}.
\end{align}

We can directly compute these and use parametric bootstrap to get their confidence intervals. We start by loading in the data and computing the MLEs for $\phi$ and $\sigma$.

In [3]:
df = pd.read_csv(os.path.join(data_path, "good_invitro_droplet_data.csv"), comment="#")

spindle_length = df["Spindle Length (um)"].values

phi_mle = np.mean(spindle_length)
sigma_mle = np.std(spindle_length)

print("φ MLE:", phi_mle, "µm")
print("σ MLE:", sigma_mle, "µm")

φ MLE: 32.86402985074626 µm
σ MLE: 4.784665043782949 µm


Now we use parametric bootstrap to compute the confidence intervals. First, we'll write functions to perform the bootstrapping.

In [4]:
rg = np.random.default_rng()

def draw_parametric_bs_reps(data, phi, sigma, size=1):
    """Parametric bootstrap replicates of parameters of
    Normal distribution."""
    bs_reps_phi = np.empty(size)
    bs_reps_sigma = np.empty(size)
    
    for i in range(size):
        bs_sample = np.random.normal(phi, sigma, size=len(data))
        bs_reps_phi[i] = np.mean(bs_sample)
        bs_reps_sigma[i] = np.std(bs_sample)
                                     
    return bs_reps_phi, bs_reps_sigma

Now we can get the bootstrap replicates.

In [5]:
bs_reps_phi, bs_reps_sigma = draw_parametric_bs_reps(
    spindle_length, phi_mle, sigma_mle, size=100000
)

With the bootstrap replicates in hand, we can compute the 95% confidence intervals using percentiles.

In [6]:
print('φ conf int:', np.percentile(bs_reps_phi, [2.5, 97.5]))
print('σ conf int:', np.percentile(bs_reps_sigma, [2.5, 97.5]))

φ conf int: [32.5033421  33.22797245]
σ conf int: [4.52513392 5.03823811]


So, we have estimated the parameters under this model.

## Computing environment

In [7]:
%load_ext watermark
%watermark -v -p numpy,pandas,jupyterlab

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 7.29.0

numpy     : 1.21.2
pandas    : 1.3.4
jupyterlab: 3.2.1

